Idea code name: `S1P10`

In [1]:
from keras.layers import Input, Dense, LSTM, PReLU, Reshape, Dropout, Activation, BatchNormalization
from keras.models import model_from_json, Model
import keras.backend as K
import keras

from s1p10_model.resnet8_body import resnet8_body

from s1p10_training_utils import gen_regressor_dataset

import json
import time

import numpy as np
import sklearn

Using TensorFlow backend.


# Global Config

In [2]:
IMAGE_SHAPE = (200, 200, 1)
NUM_LABELS = 5

# 1. Model Acrchitect

1 Image -> ResNet-8 -> Dense regressor -> 5 steering angles

## 1.2 Create Encoder
Encoder is body of ResNet-8 from `Drone-Net` since it's already trained to recognize road curve -> helpful spatial information


In [3]:
encoder = resnet8_body(IMAGE_SHAPE)
# load resnet8 weights
encoder.load_weights("./s1p10_model/named_resnet8_best_weights.h5", by_name=True)
for l in encoder.layers:
    l.trainable = False
encoder.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200, 200, 1)  0                                            
__________________________________________________________________________________________________
conv_0 (Conv2D)                 (None, 100, 100, 32) 832         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 49, 49, 32)   0           conv_0[0][0]                     
__________________________________________________________________________________________________
bn_1_a (BatchNorma

## 1.3 Create Decoder
Decoder is a seri of Dense layers with a non-activate Dense layer at the end to perform regression.

In [4]:
def dense_regressor(input_shape, num_labels):
    """
    Define classifier made of several Dense layers
    """
    X_in = Input(shape=input_shape)
    
    # 1st layer
    X = Dense(1200)(X_in)
    
    X = Activation('relu')(X)
    
    X = Dropout(0.5)(X)
    
    # 2nd layer
    X = Dense(600)(X)
    
    X = Activation('relu')(X)
    
    X = Dropout(0.5)(X)
    
    # 3rd layer
    X = Dense(200)(X)
    
    X = Activation('relu')(X)
    
    X = Dropout(0.5)(X)
    
    # 4th layer
    y = Dense(num_labels, activation=None)(X)
    
    model = Model(inputs=[X_in], outputs=[y], name='regressor')
    return model
    

In [5]:
regressor = dense_regressor((6272, ), NUM_LABELS)

regressor.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 6272)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1200)              7527600   
_________________________________________________________________
activation_8 (Activation)    (None, 1200)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1200)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 600)               720600    
_________________________________________________________________
activation_9 (Activation)    (None, 600)               0         
___________________________

## 1.4 Create Model

In [6]:
def s1p10_model(input_shape):
    """
    Define see-1-predict-10 model
    
    Input:
        input_shape (tuple): shape of input image inputted to encoder
        
    Output:
        keras.Model
    """
    X_in = Input(shape=input_shape, name="image_in")
    
    # extract feature vector
    X_feature = encoder(X_in)
    
    # apply regressor
    y = regressor(X_feature)
    
    model = Model(inputs=[X_in], outputs=[y])
    return model
        

In [7]:
model = s1p10_model(IMAGE_SHAPE)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_in (InputLayer)        (None, 200, 200, 1)       0         
_________________________________________________________________
resnet8 (Model)              (None, 6272)              309088    
_________________________________________________________________
regressor (Model)            (None, 5)                 8369405   
Total params: 8,678,493
Trainable params: 8,369,405
Non-trainable params: 309,088
_________________________________________________________________


In [8]:
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [9]:
model.compile(optimizer='Adam', loss='mse', metrics=[root_mean_squared_error])

In [10]:
class EVAMetrics(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self._data = []

    def on_epoch_end(self, batch, logs={}):
        X_val, y_val = self.validation_data[0], self.validation_data[1]
        y_predict = np.asarray(self.model.predict(X_val))

        exp_var = sklearn.metrics.explained_variance_score(y_val, y_predict, multioutput='uniform_average')
        
        self._data.append({
            'EVA': exp_var,
        })
        return

    def get_data(self):
        return self._data

# Training

In [11]:
gen_param = {'num_labels': NUM_LABELS, 
             'image_shape': IMAGE_SHAPE, 
             'num_samples': None, 
             'data_root_dir': "/home/user/Bureau/Dataset/udacity/", 
             'flip_prob': 0.5}

# X_train, y_train = gen_regressor_dataset("./s1p10_data/s1p10_CH2_002_output_training.csv", **gen_param)
X_train = np.load('./s1p10_data/CH2_training_X.npy')
y_train = np.load('./s1p10_data/CH2_training_y.npy')
# X_val, y_val = gen_regressor_dataset("./s1p10_data/s1p10_CH2_002_output_validation.csv", **gen_param)
X_val = np.load('./s1p10_data/CH2_validation_X.npy')
y_val = np.load('./s1p10_data/CH2_validation_y.npy')

In [12]:
batch_size = 100

time_str = time.strftime("%Y_%m_%d_%H_%M")
log_dir = './s1p10_logs/' + time_str
tb_callback = keras.callbacks.TensorBoard(log_dir=log_dir,  
                                          batch_size=batch_size, 
                                          update_freq='epoch')

eva_metric = EVAMetrics()
model.fit(x=X_train,
          y=y_train,
          epochs=10,
          validation_data=(X_val, y_val),
          initial_epoch=0,
          shuffle=True,
          batch_size=batch_size,
          callbacks=[tb_callback, eva_metric])
eva_metric.get_data()

Instructions for updating:
Use tf.cast instead.
Train on 18232 samples, validate on 2026 samples
Epoch 1/10
18232/18232 [==============================] - 25s 1ms/step - loss: 17.8083 - root_mean_squared_error: 2.2277 - val_loss: 0.0813 - val_root_mean_squared_error: 0.2809
Epoch 2/10
18232/18232 [==============================] - 23s 1ms/step - loss: 0.2341 - root_mean_squared_error: 0.4760 - val_loss: 0.0812 - val_root_mean_squared_error: 0.2807
Epoch 3/10
18232/18232 [==============================] - 24s 1ms/step - loss: 0.1319 - root_mean_squared_error: 0.3582 - val_loss: 0.0812 - val_root_mean_squared_error: 0.2807
Epoch 4/10
18232/18232 [==============================] - 23s 1ms/step - loss: 0.1004 - root_mean_squared_error: 0.3140 - val_loss: 0.0812 - val_root_mean_squared_error: 0.2807
Epoch 5/10
18232/18232 [==============================] - 24s 1ms/step - loss: 0.0915 - root_mean_squared_error: 0.2989 - val_loss: 0.0812 - val_root_mean_squared_error: 0.2807
Epoch 6/10
18232/

[{'EVA': -0.0014462419000519145},
 {'EVA': -3.438529685351188e-05},
 {'EVA': -1.7422276687995008e-08},
 {'EVA': -4.4408920985006264e-17},
 {'EVA': 0.0},
 {'EVA': 0.0},
 {'EVA': 0.0},
 {'EVA': -4.4408920985006264e-17},
 {'EVA': -4.4408920985006264e-17},
 {'EVA': 0.0}]

# Save weights

In [13]:
# save classifier
regressor.save_weights(log_dir + "/regressor_%s.h5" % time_str)

In [14]:
time_str

'2019_05_29_16_20'

In [15]:
# #TODO: save X_train, y_train
# np.save('./s1p10_data/CH2_training_X.npy', X_train)
# np.save('./s1p10_data/CH2_training_y.npy', y_train)

# np.save('./s1p10_data/CH2_validation_X.npy', X_val)
# np.save('./s1p10_data/CH2_validation_y.npy', y_val)